In [14]:
# If at any point you're receiving the following TypeError message:
## TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type
# Check the compatibility of your versions of numpy and pandas:
#print(np.__version__)
#print(pd.__version__)

##!pip install numpy==1.16.5 --user
##!pip install pandas==0.25.3 --user

# Import libraries and dependencies
#import panel as pn
#pn.extension('plotly')
#import plotly.express as px
import pandas as pd
#import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv
#import alpaca_trade_api as tradeapi
#from MCForecastTools import MCSimulation
import json, requests

In [15]:
#Read in the data and create a Data Frame. Drop NA as we will only be using the most recent median prices available - 2019-12. Drop unused columns, sort values, and rename '2019-12 column to 'Median Price'
br1_data = pd.read_csv('C:/Users/Myles/Documents/GitHub/Project/bcs-project1/resources/City_MedianRentalPrice_1Bedroom.csv',index_col='RegionName')
br1_df = pd.DataFrame(br1_data).dropna(axis=1).drop(['Unnamed: 0','2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11'],axis=1).sort_values('2019-12',ascending=False).rename(columns={'2019-12':'1 BR Median Price'})



br1_df.head()

,State,Metro,CountyName,SizeRank,1 BR Median Price
RegionName,,,,,
Malibu,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,768,5000.0
San Francisco,CA,San Francisco-Oakland-Hayward,San Francisco County,15,3500.0
Mountain View,CA,San Jose-Sunnyvale-Santa Clara,Santa Clara County,408,3350.0
Redwood City,CA,San Francisco-Oakland-Hayward,San Mateo County,356,3212.5
Foster City,CA,San Francisco-Oakland-Hayward,San Mateo County,678,3085.0


In [16]:
br2_data = pd.read_csv('C:/Users/Myles/Documents/GitHub/Project/bcs-project1/resources/City_MedianRentalPrice_2Bedroom.csv',index_col='RegionName')
br2_df = pd.DataFrame(br2_data).dropna(axis=1).drop(['Unnamed: 0','2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11'],axis=1).sort_values('2019-12',ascending=False).rename(columns={'2019-12':'2 BR Median Price'})

br2_df.head()

,State,Metro,CountyName,SizeRank,2 BR Median Price
RegionName,,,,,
Longboat Key,FL,North Port-Sarasota-Bradenton,Sarasota County,1265,5000.0
Siesta Key,FL,North Port-Sarasota-Bradenton,Sarasota County,1261,4600.0
San Francisco,CA,San Francisco-Oakland-Hayward,San Francisco County,15,4599.5
Palo Alto,CA,San Jose-Sunnyvale-Santa Clara,Santa Clara County,594,4400.0
Key Biscayne,FL,Miami-Fort Lauderdale-West Palm Beach,Miami-Dade County,1220,4400.0


In [17]:
br3_data = pd.read_csv('C:/Users/Myles/Documents/GitHub/Project/bcs-project1/resources/City_MedianRentalPrice_3bedroom.csv',index_col='RegionName')
br3_df = pd.DataFrame(br3_data).dropna(axis=1).drop(['Unnamed: 0','2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11'],axis=1).sort_values('2019-12',ascending=False).rename(columns={'2019-12':'3 BR Median Price'})
br3_df.head()

,State,Metro,CountyName,SizeRank,3 BR Median Price
RegionName,,,,,
Bridgehampton,NY,New York-Newark-Jersey City,Suffolk County,988,30000.0
Montauk,NY,New York-Newark-Jersey City,Suffolk County,989,21333.0
Westhampton Beach,NY,New York-Newark-Jersey City,Suffolk County,990,20250.0
Malibu,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,954,17000.0
Key Biscayne,FL,Miami-Fort Lauderdale-West Palm Beach,Miami-Dade County,980,7500.0


In [18]:
br4_data = pd.read_csv('C:/Users/Myles/Documents/GitHub/Project/bcs-project1/resources/City_MedianRentalPrice_4Bedroom.csv',index_col='RegionName')
br4_df = pd.DataFrame(br4_data).dropna(axis=1).drop(['Unnamed: 0','2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11'],axis=1).sort_values('2019-12',ascending=False).rename(columns={'2019-12':'4 BR Median Price'})

br4_df.head()

,State,Metro,CountyName,SizeRank,4 BR Median Price
RegionName,,,,,
Aspen,CO,Glenwood Springs,Pitkin County,311,50000.0
Water Mill,NY,New York-Newark-Jersey City,Suffolk County,315,40000.0
Bridgehampton,NY,New York-Newark-Jersey City,Suffolk County,313,36333.0
Sag Harbor,NY,New York-Newark-Jersey City,Suffolk County,312,30000.0
Westhampton Beach,NY,New York-Newark-Jersey City,Suffolk County,314,28166.5


In [21]:
br5_data = pd.read_csv('C:/Users/Myles/Documents/GitHub/Project/bcs-project1/resources/City_MedianRentalPrice_5BedroomOrMore.csv',index_col='RegionName')

br5_df = pd.DataFrame(br5_data).dropna(axis=1).drop(['Unnamed: 0','2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11'],axis=1).sort_values('2019-12',ascending=False).rename(columns={'2019-12':'5+ BR Median Price'})

br5_df.head()

,State,Metro,CountyName,SizeRank,2018-12,5+ BR Median Price
RegionName,,,,,,
Beverly Hills,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,48,27500.0,26800.0
Newport Beach,CA,Los Angeles-Long Beach-Anaheim,Orange County,43,12900.0,12500.0
Irvine,CA,Los Angeles-Long Beach-Anaheim,Orange County,30,5700.0,6850.0
San Diego,CA,San Diego-Carlsbad,San Diego County,6,5000.0,5500.0
Boston,MA,Boston-Cambridge-Newton,Suffolk County,13,4800.0,5000.0


In [37]:
brcombined_df = pd.concat(
    [br1_df,br2_df,br3_df,br4_df,br5_df],
    axis=0,
    join="outer"
)
brcombined_df = brcombined_df.sort_index(axis=0).fillna(0)
brcombined_df.head()

,State,Metro,CountyName,SizeRank,1 BR Median Price,2 BR Median Price,3 BR Median Price,4 BR Median Price,2018-12,5+ BR Median Price
RegionName,,,,,,,,,,
Abilene,TX,Abilene,Taylor County,211,565.0,0.0,0.0,0.0,0.0,0.0
Abilene,TX,Abilene,Taylor County,244,0.0,0.0,1295.0,0.0,0.0,0.0
Abilene,TX,Abilene,Taylor County,234,0.0,650.0,0.0,0.0,0.0,0.0
Abington,MA,Boston-Cambridge-Newton,Plymouth County,1190,0.0,2035.0,0.0,0.0,0.0,0.0
Acton,MA,Boston-Cambridge-Newton,Middlesex County,1117,0.0,1795.0,0.0,0.0,0.0,0.0
